In [30]:
import os
import shutil
import random
import json
from tqdm import tqdm

## Настройки

In [31]:
dataset_root = "dataset_yolo"
images_dir = "data"
annotations_file = "data/_annotations.coco.json"
train_ratio = 0.8
random.seed(42)

## Папки

In [32]:
for folders in ["train", "val"]:
    os.makedirs(os.path.join(dataset_root, folders, "images"), exist_ok=True)
    os.makedirs(os.path.join(dataset_root, folders, "labels"), exist_ok=True)

## COCO

In [33]:
with open(annotations_file, 'r') as f:
    coco = json.load(f)

images = coco["images"]
annotations = coco["annotations"]
categories = coco["categories"]
category_id_map = {cat["id"]: idx for idx, cat in enumerate(categories)}



## РАЗБИВАЕМ НА ОБУЧЕНИЕ И ВАЛИДАЦИЮ

In [34]:
random.shuffle(images)
n_train = int(len(images) * train_ratio)
train_images = images[:n_train]
val_images = images[n_train:]

train_img_ids = {img["id"] for img in train_images}
val_img_ids = {img["id"] for img in val_images}


# ГРУППИРУЕМ АННОТАЦИИ

In [35]:
def group_annotations(annotations, image_ids):
    grouped = {}
    for ann in annotations:
        if ann["image_id"] in image_ids:
            if len(ann["bbox"]) == 4 and ann["bbox"][2] > 0 and ann["bbox"][3] > 0:
                grouped.setdefault(ann["image_id"], []).append(ann)
            else:
                print(f"Invalid bbox in annotation: {ann}")
    return grouped

train_anns = group_annotations(annotations, train_img_ids)
val_anns = group_annotations(annotations, val_img_ids)


## КОНВЕРТАЦИЯ COCO -> YOLO

In [36]:
# конвертация bbox
def convert_bbox(size, bbox):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x, y, w, h = bbox
    x_center = x + w / 2.0
    y_center = y + h / 2.0
    return [x_center * dw, y_center * dh, w * dw, h * dh]

# создание labels
def save_yolo_labels(images, ann_map, split):
    empty_files = 0
    for img in tqdm(images, desc=f"Processing {split}"):
        file_name = img["file_name"]
        img_id = img["id"]
        width = img["width"]
        height = img["height"]
        yolo_lines = []

        for ann in ann_map.get(img_id, []):
            orig_cat_id = ann["category_id"]
            if orig_cat_id not in category_id_map:
                continue
            yolo_cat_id = category_id_map[orig_cat_id]
            try:
                bbox = convert_bbox((width, height), ann["bbox"])
                bbox_str = " ".join([f"{x:.6f}" for x in bbox])
                yolo_lines.append(f"{yolo_cat_id} {bbox_str}")
            except Exception as e:
                continue

        label_path = os.path.join(dataset_root, split, "labels", os.path.splitext(file_name)[0] + ".txt")
        with open(label_path, "w") as f:
            f.write("\n".join(yolo_lines))
        
        if not yolo_lines:
            empty_files += 1

        # Копируем изображение
        src_path = os.path.join(images_dir, file_name)
        dst_path = os.path.join(dataset_root, split, "images", file_name)
        if os.path.exists(src_path):
            shutil.copy2(src_path, dst_path)
    

In [37]:
save_yolo_labels(train_images, train_anns, "train")
save_yolo_labels(val_images, val_anns, "val")

Processing val: 100%|██████████| 214/214 [00:00<00:00, 317.78it/s]


## Обучение

In [38]:
!yolo task=detect mode=train model=yolov8n.pt data=dataset.yaml epochs=10 imgsz=640 batch=8

Ultralytics 8.3.127 🚀 Python-3.13.2 torch-2.7.0 CPU (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, re

In [ ]:
# !grep "^5 " dataset_yolo/train/labels/*.txt